In [1]:
from collections import defaultdict
import pickle
import pandas as pd
from toolz import interleave
import numpy as np


# Load JTE607 data from both backbones individually
# Filter based off of DMSO reads per variant
# Adds one to counts if there is a missing PAS from the list generated from DMSO. This is for log ratio.

# L3 loading and counting reads, filter away variants with < 50 reads in DMSO
L3_shared_5p = "GCGAATTGGAGCTCTTCTTTTTGTCACTTGAAAAACATGTAAAAATAATGTACTAGGAGACACTTTCAATAAA"
L3_shared_3p = "TCGGGTGATTATTTACCCCCCACCCTTGCCGTCTGCGAGAATTCGAT"
parsed_L3_cleaved_output_dir = "/JTE-607/Analysis/parsed_L3_input_RNA_clusterPASRandom_bbmerge_xloose/parsed_L3_cleaved_RNA_multimapping_mincov1_preload_bbmerge_xloose_H1shortN4indel/collapsed/"
polyA_L3_DMSO_cleaved_pickle = parsed_L3_cleaved_output_dir + "L3_DMSO_polya_pos_dict.pickle"
polyA_L3_0p5uM_cleaved_pickle = parsed_L3_cleaved_output_dir + "L3_0p5uM_polya_pos_dict.pickle"
polyA_L3_2p5uM_cleaved_pickle = parsed_L3_cleaved_output_dir + "L3_2p5uM_polya_pos_dict.pickle"
polyA_L3_12p5uM_cleaved_pickle = parsed_L3_cleaved_output_dir + "L3_12p5uM_polya_pos_dict.pickle"

L3_polyA_pos_pickle_dict = {"L3_DMSO": polyA_L3_DMSO_cleaved_pickle,\
              "L3_0p5uM": polyA_L3_0p5uM_cleaved_pickle,\
             "L3_2p5uM": polyA_L3_2p5uM_cleaved_pickle,\
              "L3_12p5uM": polyA_L3_12p5uM_cleaved_pickle}

L3_PAS_read_counts = defaultdict(int)
polya_pos_dict = pickle.load(open(polyA_L3_DMSO_cleaved_pickle, "rb"))
data_name = "L3_DMSO"
for curr_PAS, curr_polya_pos_count_dict in polya_pos_dict.items():
    curr_total_reads = sum(curr_polya_pos_count_dict.values())
    curr_PAS = curr_PAS.strip()
    if curr_total_reads >= 50:
        L3_PAS_read_counts[curr_PAS] += curr_total_reads

# SVLst loading and counting reads, filter away variants with < 100 reads in DMSO
SVLst_shared_5p = "GCGAATTGGAGCTCATGCTTTATTTGTGAAATTTGTGATGCTATTGCTTTATTTGTAACCATTATAAGCTGCAATAAA"
SVLst_shared_3p = "ATTTTATGTTTCAGGTTCAGGGGGAGGTGTGGGAGGTTTTTTAAAGCAAGTAGAATTCGAT"
parsed_SVLst_cleaved_output_dir = "/JTE-607/Analysis/parsed_SVLst_input_RNA_clusterPASRandom_bbmerge_xloose/parsed_SVLst_cleaved_RNA_multimapping_mincov1_preload_bbmerge_xloose_H1shortN4indel/collapsed/"
polyA_SVLst_DMSO_cleaved_pickle = parsed_SVLst_cleaved_output_dir + "SVLst_DMSO_polya_pos_dict.pickle"
polyA_SVLst_0p5uM_cleaved_pickle = parsed_SVLst_cleaved_output_dir + "SVLst_0p5uM_polya_pos_dict.pickle"
polyA_SVLst_2p5uM_cleaved_pickle = parsed_SVLst_cleaved_output_dir + "SVLst_2p5uM_polya_pos_dict.pickle"
polyA_SVLst_12p5uM_cleaved_pickle = parsed_SVLst_cleaved_output_dir + "SVLst_12p5uM_polya_pos_dict.pickle"

SVLst_polyA_pos_pickle_dict = {"SVLst_DMSO": polyA_SVLst_DMSO_cleaved_pickle,\
              "SVLst_0p5uM": polyA_SVLst_0p5uM_cleaved_pickle,\
             "SVLst_2p5uM": polyA_SVLst_2p5uM_cleaved_pickle,\
              "SVLst_12p5uM": polyA_SVLst_12p5uM_cleaved_pickle}

SVLst_PAS_read_counts = defaultdict(int)
SVLst_total_reads = 0
polya_pos_dict = pickle.load(open(polyA_SVLst_DMSO_cleaved_pickle, "rb"))
data_name = "SVLst_DMSO"
for curr_PAS, curr_polya_pos_count_dict in polya_pos_dict.items():
    curr_total_reads = sum(curr_polya_pos_count_dict.values())
    curr_PAS = curr_PAS.strip()
    if curr_total_reads >= 50:
        SVLst_PAS_read_counts[curr_PAS] += curr_total_reads

# combine L3 and SVLst, calculate percentages within each dataset
L3_PAS_percents_dict = {"L3_DMSO": None,\
              "L3_0p5uM": None,\
             "L3_2p5uM": None,\
              "L3_12p5uM": None}
SVLst_PAS_percents_dict = {"SVLst_DMSO": None,\
              "SVLst_0p5uM": None,\
             "SVLst_2p5uM": None,\
              "SVLst_12p5uM": None}

all_PAS = set()
for data_name, pickle_name in L3_polyA_pos_pickle_dict.items():
    polya_pos_dict = pickle.load(open(pickle_name, "rb"))
    PAS_read_counts = defaultdict(int)
    if set(polya_pos_dict.keys()).intersection(set(L3_PAS_read_counts.keys())) != set(L3_PAS_read_counts.keys()):
        add_one = True
        for curr_PAS in set(L3_PAS_read_counts.keys()) - set(polya_pos_dict.keys()):
            PAS_read_counts[curr_PAS] += 1
    else:
        add_one = False
    print("%s add_one = %s" % (data_name, add_one))
    for curr_PAS, curr_polya_pos_count_dict in polya_pos_dict.items():
        if curr_PAS in L3_PAS_read_counts:
            curr_total_reads = sum(curr_polya_pos_count_dict.values())
            #curr_PAS = L3_shared_5p + curr_PAS + L3_shared_3p
            PAS_read_counts[curr_PAS] += curr_total_reads
            if add_one:
                PAS_read_counts[curr_PAS] += 1
            all_PAS.add(curr_PAS)
    total_reads = float(sum(PAS_read_counts.values()))  # float for division later
    percentage_dict = {curr_PAS:(curr_read_count/total_reads) for curr_PAS, curr_read_count in PAS_read_counts.items()}
    df = pd.DataFrame(list(percentage_dict.items()),columns = ['PAS', data_name.split("_")[1] + '_percent'])
    df["Background"] = "L3"
    L3_PAS_percents_dict[data_name] = df

for data_name, pickle_name in SVLst_polyA_pos_pickle_dict.items():
    polya_pos_dict = pickle.load(open(pickle_name, "rb"))
    PAS_read_counts = defaultdict(int)
    if set(polya_pos_dict.keys()).intersection(set(SVLst_PAS_read_counts.keys())) != set(SVLst_PAS_read_counts.keys()):
        add_one = True
        for curr_PAS in set(SVLst_PAS_read_counts.keys()) - set(polya_pos_dict.keys()):
            PAS_read_counts[curr_PAS] += 1
    else:
        add_one = False
    print("%s add_one = %s" % (data_name, add_one))
    for curr_PAS, curr_polya_pos_count_dict in polya_pos_dict.items():
        if curr_PAS in SVLst_PAS_read_counts:
            curr_total_reads = sum(curr_polya_pos_count_dict.values())
            #curr_PAS = SVLst_shared_5p + curr_PAS + SVLst_shared_3p
            PAS_read_counts[curr_PAS] += curr_total_reads
            if add_one:
                PAS_read_counts[curr_PAS] += 1
            all_PAS.add(curr_PAS)
    total_reads = float(sum(PAS_read_counts.values()))  # float for division later
    percentage_dict = {curr_PAS:(curr_read_count/total_reads) for curr_PAS, curr_read_count in PAS_read_counts.items()}
    df = pd.DataFrame(list(percentage_dict.items()),columns = ['PAS', data_name.split("_")[1] + '_percent'])
    df["Background"] = "SVLst"
    SVLst_PAS_percents_dict[data_name] = df

print(len(all_PAS))

merged_PAS_percentage_L3 = None
for data_name, curr_df in L3_PAS_percents_dict.items():
    if merged_PAS_percentage_L3 is None:
        merged_PAS_percentage_L3 = curr_df
    else:
        merged_PAS_percentage_L3 = merged_PAS_percentage_L3.merge(curr_df.loc[:, curr_df.columns != "Background"], how='outer', on="PAS")
merged_PAS_percentage_L3 = merged_PAS_percentage_L3.fillna(0)
print(merged_PAS_percentage_L3.shape)
print("Unsorted merged_PAS_percentage_L3: ", merged_PAS_percentage_L3)
merged_PAS_percentage_L3 = merged_PAS_percentage_L3.set_index("PAS")
merged_PAS_percentage_L3.reset_index(level=0, inplace=True)
merged_PAS_percentage_L3 = merged_PAS_percentage_L3.sort_values(by=["DMSO_percent"], ascending=False)
print("Sorted merged_PAS_percentage_L3: ", merged_PAS_percentage_L3)

# split into even and odd dataframes, and then merge together
merged_PAS_percentage_L3_odd = merged_PAS_percentage_L3.iloc[1::2]
merged_PAS_percentage_L3_even = merged_PAS_percentage_L3.iloc[::2]
merged_PAS_percentage_L3 = pd.concat([merged_PAS_percentage_L3_odd, merged_PAS_percentage_L3_even]).reset_index(drop=True)
print("Odd even merged_PAS_percentage_L3: ", merged_PAS_percentage_L3)

merged_PAS_percentage_SVLst = None
for data_name, curr_df in SVLst_PAS_percents_dict.items():
    if merged_PAS_percentage_SVLst is None:
        merged_PAS_percentage_SVLst = curr_df
    else:
        merged_PAS_percentage_SVLst = merged_PAS_percentage_SVLst.merge(curr_df.loc[:, curr_df.columns != "Background"], how='outer', on="PAS")
merged_PAS_percentage_SVLst = merged_PAS_percentage_SVLst.fillna(0)
print(merged_PAS_percentage_SVLst.shape)
#print("Unsorted merged_PAS_percentage_SVLst: ", merged_PAS_percentage_SVLst)
merged_PAS_percentage_SVLst = merged_PAS_percentage_SVLst.set_index("PAS")
merged_PAS_percentage_SVLst.reset_index(level=0, inplace=True)
merged_PAS_percentage_SVLst = merged_PAS_percentage_SVLst.sort_values(by=["DMSO_percent"], ascending=False)
#print("Sorted merged_PAS_percentage_SVLst: ", merged_PAS_percentage_SVLst)

# split into even and odd dataframes, and then merge together
merged_PAS_percentage_SVLst_odd = merged_PAS_percentage_SVLst.iloc[1::2]
merged_PAS_percentage_SVLst_even = merged_PAS_percentage_SVLst.iloc[::2]
merged_PAS_percentage_SVLst = pd.concat([merged_PAS_percentage_SVLst_odd, merged_PAS_percentage_SVLst_even]).reset_index(drop=True)
print("Odd even merged_PAS_percentage_SVLst: ", merged_PAS_percentage_SVLst)

merged_PAS_percentage = pd.DataFrame(interleave([merged_PAS_percentage_L3.values, merged_PAS_percentage_SVLst.values]), \
                                     columns=merged_PAS_percentage_L3.columns)
print("Interleaved merged_PAS_percentage: ", merged_PAS_percentage)

del merged_PAS_percentage_SVLst, merged_PAS_percentage_L3, SVLst_PAS_percents_dict, L3_PAS_percents_dict, all_PAS, polya_pos_dict

L3_DMSO add_one = False
L3_0p5uM add_one = False
L3_2p5uM add_one = True
L3_12p5uM add_one = True
SVLst_DMSO add_one = False
SVLst_0p5uM add_one = False
SVLst_2p5uM add_one = False
SVLst_12p5uM add_one = True
261316
(158298, 6)
Unsorted merged_PAS_percentage_L3:                                PAS  DMSO_percent Background  0p5uM_percent  \
0       GACTTCGGTGAGCAACGCGGGTCCG      0.000006         L3       0.000006   
1       GGTAGATTACGTCATTGTGTCTTTG      0.000008         L3       0.000009   
2       TGGTTGAACTGTCAAACTTGGTTTC      0.000007         L3       0.000010   
3       GTTTCGGCTTCTCATCCACTGTGGA      0.000004         L3       0.000005   
4       GGACTTAGCTGTCAGACTCATTGCG      0.000010         L3       0.000011   
...                           ...           ...        ...            ...   
158293  TTATATACTGTTCATTCAGCGTATT      0.000004         L3       0.000006   
158294  GGCTGACTGGGGTAAAGCGTTGGTA      0.000004         L3       0.000002   
158295  CTAGGAGCAATTCAAATCTGAAATC      0.00

In [2]:
"""
Change to requested ratio of drug / DMSO
"""

# need to avoid log(0)
zero_drug_plus = merged_PAS_percentage[["0p5uM_percent","2p5uM_percent","12p5uM_percent"]]
merged_ratios = zero_drug_plus.div(merged_PAS_percentage["DMSO_percent"], axis=0)
merged_ratios = np.log(merged_ratios)
merged_ratios["PAS"] = merged_PAS_percentage["PAS"]
merged_ratios["Background"] = merged_PAS_percentage["Background"]
merged_PAS_percentage_DMSO = merged_PAS_percentage["DMSO_percent"]

print("merged_ratios:\n", merged_ratios)

del merged_PAS_percentage

merged_ratios:
         0p5uM_percent  2p5uM_percent  12p5uM_percent  \
0           -0.063896       0.214028        0.408453   
1            0.158997       0.248974        0.382862   
2           -0.169079       0.045483        0.017941   
3           -0.011128      -0.358545       -0.694788   
4           -0.263721      -0.588781       -1.410902   
...               ...            ...             ...   
261311      -0.003815       0.308947       -1.034874   
261312      -0.194870      -0.330133       -0.287659   
261313       0.792517       0.388989       -0.141056   
261314      -0.048267      -0.278840       -0.141056   
261315       0.038745       0.021264        0.351421   

                              PAS Background  
0       GCTTGGAAGATCTACACGCGTGTAT         L3  
1       TGTTTGTTAATCAAGATGAATGAGC      SVLst  
2       CAGGAGTGAGAACATTAATCCTAAC         L3  
3       AGGCGTGTTGCCAATGTCGTCTGAG      SVLst  
4       GTACGATCGATGTATCGTTTCGTTG         L3  
...                          

In [3]:
descending_merged_ratios = merged_ratios.sort_values(by=["12p5uM_percent"], ascending=False)
print("descending_merged_ratios: ", descending_merged_ratios)

descending_merged_ratios:          0p5uM_percent  2p5uM_percent  12p5uM_percent  \
103499       0.682352       0.902100        1.726436   
15700        0.702510       1.213874        1.652243   
153730       0.731892       1.029493        1.596215   
56229        0.556066       0.536265        1.587475   
134572       0.633878       1.091118        1.494962   
...               ...            ...             ...   
228546      -0.662371      -1.107789       -3.463210   
227452      -0.821220      -2.422820       -3.478958   
78258       -0.402271      -0.446563       -3.539583   
218442      -0.767731      -1.126138       -3.568571   
210902      -0.312995      -1.308460       -3.663881   

                              PAS Background  
103499  CATGTGAGTGCTACTGTCGGTGGGC      SVLst  
15700   ACGATGCGCTCTCAAGCGCTCAGTA         L3  
153730  CACGGCGGATCCTAATATGACAGAT         L3  
56229   CAAACCCAAAGTAATGCCGGACAGA      SVLst  
134572  CGGTATGGATCCTATGTGTGTCGTA         L3  
...               

In [4]:
L3_merged_ratios = descending_merged_ratios.loc[merged_ratios['Background'] == "L3"]
SVLst_merged_ratios = descending_merged_ratios.loc[merged_ratios['Background'] == "SVLst"]

print("L3_merged_ratios = ", L3_merged_ratios)
print("SVLst_merged_ratios = ", SVLst_merged_ratios)

L3_merged_ratios =          0p5uM_percent  2p5uM_percent  12p5uM_percent  \
15700        0.702510       1.213874        1.652243   
153730       0.731892       1.029493        1.596215   
134572       0.633878       1.091118        1.494962   
255938       0.337396       1.049430        1.484297   
18392        0.727457       0.997659        1.477990   
...               ...            ...             ...   
228546      -0.662371      -1.107789       -3.463210   
227452      -0.821220      -2.422820       -3.478958   
78258       -0.402271      -0.446563       -3.539583   
218442      -0.767731      -1.126138       -3.568571   
210902      -0.312995      -1.308460       -3.663881   

                              PAS Background  
15700   ACGATGCGCTCTCAAGCGCTCAGTA         L3  
153730  CACGGCGGATCCTAATATGACAGAT         L3  
134572  CGGTATGGATCCTATGTGTGTCGTA         L3  
255938  GCCAAGGGATCGTAGTTGGAAATGC         L3  
18392   GAGTTCGCCGATCATGGGTCATGGT         L3  
...                      

In [5]:
L3_merged_ratios_top1k_resistant = L3_merged_ratios.head(1000)
L3_merged_ratios_top1k_sensitive = L3_merged_ratios.tail(1000)

SVLst_merged_ratios_top1k_resistant = SVLst_merged_ratios.head(1000)
SVLst_merged_ratios_top1k_sensitive = SVLst_merged_ratios.tail(1000)

output_dir = "/JTE-607/Analysis/combined_backbones_mincov1_H1shortN4indel/"

L3_merged_ratios_top1k_resistant.to_csv(output_dir + "L3_merged_ratios_top_1k_resistant_mincov50_allCutSites.txt", sep="\t")
L3_merged_ratios_top1k_sensitive.to_csv(output_dir + "L3_merged_ratios_top_1k_sensitive_mincov50_allCutSites.txt", sep="\t")

SVLst_merged_ratios_top1k_resistant.to_csv(output_dir + "SVLst_merged_ratios_top_1k_resistant_mincov50_allCutSites.txt", sep="\t")
SVLst_merged_ratios_top1k_sensitive.to_csv(output_dir + "SVLst_merged_ratios_top_1k_sensitive_mincov50_allCutSites.txt", sep="\t")